In [3]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../..'))
	print(os.getcwd())
except:
	pass


/home/web


In [28]:
from data_science.probability import normal_cdf, inverse_normal_cdf
import math
import random


def normal_approximation_to_binominal(n, p):
    """
    Аппроксимация биноминальной случайной величины нормальным распределением;
    Находит mu и sigma, которые соответствуют binominal(n, p)
    """
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma


# Вероятность, что значение нормальной случайной величины
# лежит ниже порогового значения
normal_probability_below = normal_cdf


def normal_probability_above(lo, mu=0, sigma=1):
    """Оно выше порогового значения если оно не ниже его"""
    return 1 - normal_cdf(lo, mu, sigma)


def normal_probability_between(lo, hi, mu=0, sigma=1):
    """Оно лежит между, если оно между, если оно меньше hi, но не ниже lo"""
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)


def normal_probability_outside(lo, hi, mu=0, sigma=1):
    """Оно лежит за пределами, если оно не внутри"""
    return 1 - normal_probability_between(lo, hi, mu, sigma)


def normal_upper_bound(probability, mu=0, sigma=1):
    """
    Верхняя граница нормальной величины
    Возвращает z, для которого P(Z <= z) = probability
    """
    return inverse_normal_cdf(probability, mu, sigma)


def normal_lower_bound(probability, mu=0, sigma=1):
    """
    Нижняя граница нормально величины
    Возвращает z, для которого P(Z >= z) = probability
    """
    return inverse_normal_cdf(1 - probability, mu, sigma)


def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """
    Двусторонние границы нормальной величны
    Возвращает симметричные (вокруг своего значения) границы,
    в пределах которой находится указанная вероятность
    """
    tail_probability = (1 - probability) / 2

    # Верхняя граница должна иметь значение хвостовой вероятности
    # tail_probability ниже ее
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # Нижняя граница должна иметь значение хвостовой вероятности
    # tail_probability ниже её
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound


def two_sided_p_value(x, mu=0, sigma=1):
    """Двустороннее p - значение"""
    if x >= mu:
        # Если х больше среднего значения, то значения в хвосте больше х
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # Если х меньше среднего значения, то значения в хвосте меньше х
        return 2 * normal_probability_below(x, mu, sigma)

def run_experiment():
    """
    Провести эксперимент: бросить уравновешенную монету 1000 раз
    True - орлы, False - решки
    """
    return [random.random() < 0.5 for _ in range(1000)]


def reject_fairness(experiment):
    """
    Отвергнуть уравновешенность монеты
    Используя 5% уровни значимости
    """
    num_heads = len([flip for flip in experiment if flip]) # число орлов
    return num_heads < 469 or num_heads > 531


def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma


def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)


In [5]:
mu_0, sigma_0 = normal_approximation_to_binominal(1000, 0.5)

In [6]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)
# Двусторонние границы нормальной случайной величины

(469.01026640487555, 530.9897335951244)

In [7]:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
# 95% границы при условии что p = 0.5

In [8]:
mu_1, sigma_1 = normal_approximation_to_binominal(1000, 0.55)
# Фактические sigma и mu при P = 0.55

In [9]:
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability  # 0.887
# ошибка второго рода означает: не удалось отклонить нулевую гипотезу; 
# это происходит, когда x всё еще внутри первоначального интервала
power

0.886548001295367

In [10]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
# Верхняя граница нормальной случайной величины
# = 526 (< 531 поскольку нужно больше вероятности в верхнем хвосте)

In [11]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
1 - type_2_probability # power

0.9363794803307173

In [13]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [16]:
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1
extreme_value_count / 100000

0.06214

In [17]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [18]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [19]:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

In [20]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

In [23]:
p_hat = 525 / 1000  # пропорция орлов в выборке (р "с крышкой")
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
sigma

0.015791611697353755

In [24]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

In [25]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095927295919, 0.5708904072704082)

In [27]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])
num_rejections

46

In [29]:
z = a_b_test_statistic(1000, 200, 1000, 180)
two_sided_p_value(z)

0.2541419765422359

In [30]:
z = a_b_test_statistic(1000, 200, 1000, 150)
two_sided_p_value(z)

0.003189699706216853